# A Multi Target Random Forest Regression (MTRT)

In this section we develop a multi target random forest regression for the prediction of a minimum and maximum price of laptop models.


## 1. Import Statements


In [8]:
import pandas as pd
import numpy as np
import math
import io

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pydotplus

from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestRegressor 

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.tree import plot_tree
from sklearn.externals.six import StringIO

from IPython.display import SVG
from graphviz import Source
from IPython.display import display

## Edit Print options
desired_width = 500
pd.set_option('display.width', desired_width)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 600)

## 2. Dealing with missing values



In [9]:
## Importing the preprocessed data
missing_values = ["n/a", "na", "--","NAN"," ","nan","NaN",""]
data = pd.read_csv('datafile2.csv',na_values = missing_values)
data2 = pd.read_csv('datafile2_test.csv',na_values = missing_values)

## Removing missing values
# data.dropna()

## Just a temporary dataset to test the capabilities of the model
temp_data = data[['brand','base_name','os_details','gpu_brand','cpu_brand','cpu_type_name','gpu_type','screen_surface','screen_size','pixels_x',
                  'touchscreen','discrete_gpu','gpu','ram','ssd','detachable_keyboard','weight',
                  'cpu_benchmark','gpu_benchmark','cpu_GHZ','cpu_core','threading','pc_name','min_price','max_price']]

test_data = data2[['brand','base_name','os_details','gpu_brand','cpu_brand','cpu_type_name','gpu_type','screen_surface','screen_size','pixels_x',
                   'touchscreen','discrete_gpu','gpu','ram','ssd', 'detachable_keyboard','weight',
                   'cpu_benchmark','gpu_benchmark','cpu_GHZ','cpu_core','threading','pc_name']]

concat_df = pd.concat([temp_data , test_data])
concat_df.to_csv("/Users/Simon/Documents/GitHub/adana123/concat_df1.csv",header=True,index=True)

##### TRAINING DATA #####
## Filling in missing values: CATEGORICAL & BINARY VARIABLES
detach_key = int(temp_data.loc[:,'detachable_keyboard'].mode())
temp_data.loc[:,'detachable_keyboard'] = temp_data.loc[:,'detachable_keyboard'].fillna(0)

temp_data.loc[:,'gpu_brand'] = temp_data.loc[:,'gpu_brand'].fillna("Unknown", inplace = False) 
temp_data['cpu_type_name'] = temp_data.apply(lambda row: row['cpu_brand'] if pd.isnull(row['cpu_type_name']) 
                                             else row['cpu_type_name'], axis=1)
temp_data['gpu_type'] = temp_data['gpu_type'].fillna("No GPU", inplace = False) 
temp_data.loc[:,'screen_surface'] = temp_data.loc[:,'screen_surface'].fillna("Unknown", inplace = False) 
temp_data['os_details'] = temp_data['os_details'].fillna("Unknown", inplace = False) 
temp_data['cpu_core'] = temp_data['cpu_core'].fillna(temp_data['cpu_core'].mode())

# ## Filling in missing values: NUMERICAL VARIABLES
temp_data['weight'] = temp_data['weight'].fillna(temp_data['weight'].mean())
temp_data['cpu_benchmark'] = temp_data['cpu_benchmark'].fillna(temp_data['cpu_benchmark'].mean()) 
temp_data['gpu_benchmark'] = temp_data['gpu_benchmark'].fillna(temp_data['gpu_benchmark'].mean())
temp_data['cpu_GHZ'] = temp_data['cpu_GHZ'].fillna(temp_data['cpu_GHZ'].mean())
pixel_mode = int(temp_data.loc[:,'pixels_x'].mode())
temp_data['pixels_x'] = temp_data['pixels_x'].fillna(pixel_mode) #Modus! Most frequent number! (median)

## Showing the data
# data.head()

########################################################################################################

##### TRAINING & TESTING DATA #####
## Filling in missing values: CATEGORICAL & BINARY VARIABLES
detach_key = int(temp_data.loc[:,'detachable_keyboard'].mode())
concat_df.loc[:,'detachable_keyboard'] = concat_df.loc[:,'detachable_keyboard'].fillna(detach_key)
# print(concat_df['detachable_keyboard'])
concat_df['gpu_brand'] = concat_df['gpu_brand'].fillna("Unknown", inplace = False) 
concat_df['cpu_type_name'] = concat_df.apply(lambda row: row['cpu_brand'] if pd.isnull(row['cpu_type_name']) 
                                             else row['cpu_type_name'], axis=1)

concat_df['gpu_type'] = concat_df['gpu_type'].fillna("No GPU", inplace = False)
concat_df['gpu'] = concat_df['gpu'].fillna("No GPU", inplace = False) 
concat_df['screen_surface'] = concat_df['screen_surface'].fillna("Unknown", inplace = False) 
concat_df['os_details'] = concat_df['os_details'].fillna("Unknown", inplace = False) 
concat_df['cpu_core'] = concat_df['cpu_core'].fillna(temp_data['cpu_core'].mode())

## Filling in missing values: NUMERICAL VARIABLES
concat_df['weight'] = concat_df['weight'].fillna(temp_data['weight'].mean())
concat_df['cpu_benchmark'] = concat_df['cpu_benchmark'].fillna(temp_data['cpu_benchmark'].mean()) 
concat_df['gpu_benchmark'] = concat_df['gpu_benchmark'].fillna(temp_data['gpu_benchmark'].mean())
concat_df['cpu_GHZ'] = concat_df['cpu_GHZ'].fillna(temp_data['cpu_GHZ'].mean())

pixel_mode = int(temp_data.loc[:,'pixels_x'].mode())
concat_df.loc[:,'pixels_x'] = concat_df.loc[:,'pixels_x'].fillna(pixel_mode)



temp_data.to_csv("/Users/Simon/Documents/GitHub/adana123/temp_data.csv",header=True,index=True)
concat_df.to_csv("/Users/Simon/Documents/GitHub/adana123/concat_df2.csv",header=True,index=True)


#### Mean only from training set!

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#in

## 3. Feature Importance (Selection)

## 4. Splitting Data & Fitting Data

In [11]:
#### Training Data ####
## Seperating categorical - binary - numerical variables
X_cat = temp_data[['cpu_core','brand','cpu_brand','gpu_brand','gpu','cpu_type_name','screen_surface']]#,'gpu_type','os_details','base_name'
X_bin = temp_data[['touchscreen','discrete_gpu','detachable_keyboard','threading']]
X_num = temp_data[['screen_size','ram','ssd','weight','cpu_GHZ','pixels_x','cpu_benchmark','gpu_benchmark']]

## Creating dummy variables:
X_cat_dummies = pd.get_dummies(X_cat, drop_first=True)

## Merging input data
X = pd.concat([X_cat_dummies,X_bin,X_num], axis=1)
# X = pd.concat([X_cat,X_bin,X_num], axis=1)
# X = pd.concat([X_cat_dummies,X_num], axis=1)
# X = X_cat_dummies

## Defining output data
Ya = temp_data[['min_price','max_price']]
# Yb = temp_data[['min_price','diff_price']]

########################################################################################
## Seperating categorical - binary - numerical variables
X_cat_test = concat_df[['cpu_core','brand','cpu_brand','gpu_brand','gpu','cpu_type_name','screen_surface']]#'pc_name','gpu_type','os_details','base_name'
X_bin_test = concat_df[['touchscreen','discrete_gpu','detachable_keyboard','threading']]
X_num_test = concat_df[['screen_size','ram','ssd','weight','cpu_GHZ','pixels_x','cpu_benchmark','gpu_benchmark']]

## Creating dummy variables:
X_cat_dummies_test = pd.get_dummies(X_cat_test, drop_first=True)

## Merging input data
X_full = pd.concat([X_cat_dummies_test,X_bin_test,X_num_test], axis=1)
X_train = X_full.head(len(data))
X_test = X_full.tail(len(data2))

########################################################################################
## Splitting the dataset into train and test sets
# Predict Minimum and Maximum prices
X_train_a, X_test_a, Y_train_a, Y_test_a = train_test_split(X, Ya, test_size=0.33, random_state=42)
# Predict Minimum price and deviation from this price
# X_train_b, X_test_b, Y_train_b, Y_test_b = train_test_split(X, Yb, test_size=0.33, random_state=42)

# trainingdata = pd.concat([X_train_a, Y_train_a], axis=1, sort=False)
# testingdata = pd.concat([X_test_a, Y_test_a], axis=1, sort=False)

# trainingdata.to_csv("/Users/Simon/Documents/GitHub/adana123/trainingdata.csv",header=True,index=True)
# testingdata.to_csv("/Users/Simon/Documents/GitHub/adana123/testingdata.csv",header=True,index=True)

## 2 models
Model1 = RandomForestRegressor(n_estimators = 400,criterion='mae',random_state = 1)
Model1.fit(X_train_a,Y_train_a)

# Model2 = RandomForestRegressor(n_estimators = 200,criterion='mae',random_state = 0)
# Model2.fit(X_train_b,Y_train_b)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=400, n_jobs=None, oob_score=False,
                      random_state=1, verbose=0, warm_start=False)

## 5. Measuring Mean Summed Absolute Error

### A: MSAE for predicting minimum and maximum  price

In [12]:
## Predicting the target values
Y_pred = Model1.predict(X_test_a)

## Calculating the score
MAE = mean_absolute_error(Y_test_a, Y_pred, multioutput='raw_values')
print(MAE[0]+MAE[1])

## Turn DF to array
Y_test_array = Y_test_a.to_numpy()

Sum_error = 0
Y_pred_min_array = []
for i in range(len(Y_pred)):
#     print(Y_pred[i],Y_test_array[i])
    Y_pred_min = Y_pred[i][0]
    Y_pred_max = Y_pred[i][1]
    Y_test_min = Y_test_array[i][0]
    Y_test_max = Y_test_array[i][1]
    
    Y_pred_min_array.append(Y_pred_min)
    
    Error_min = abs(Y_pred_min - Y_test_min)
    Error_max = abs(Y_pred_max - Y_test_max)
    Total_error = Error_min + Error_max
    
    Sum_error = Sum_error + Total_error
    counter = i

Avg_error = Sum_error / counter
MSAE = Avg_error
print(MSAE)



313.4823770710061
315.3483436011905


### B: MSAE for predicting minimum price and price difference

**Result:** No significant improvement was obtained. This is as expected because the regression tree considers the two outputs, min and max price, at the same time. They are not considered independent and the splits are based on improving both outputs.

In [5]:
# Y_pred = Model2.predict(X_test_b)

# ## Turn DF to array
# Y_test_array = Y_test_b.to_numpy()

# Sum_error = 0
# Y_pred_min_array = []
# for i in range(len(Y_pred)):
# #     print(Y_pred[i],Y_test_array[i])
#     Y_pred_min = Y_pred[i][0]
#     Y_pred_max = Y_pred_min + Y_pred[i][1]
#     Y_test_min = Y_test_array[i][0]
#     Y_test_max = Y_test_min + Y_test_array[i][1]
    
#     Y_pred_min_array.append(Y_pred_min)
    
#     Error_min = abs(Y_pred_min - Y_test_min)
#     Error_max = abs(Y_pred_max - Y_test_max)
#     Total_error = Error_min + Error_max
    
#     Sum_error = Sum_error + Total_error
#     counter = i

# Avg_error = Sum_error / counter
# MSAE = Avg_error
# print('The MSAE = ', round(MSAE,3))



## 6. Visualization of Model and Data

In [6]:
## Printing the decision tree
# FEATURE_NAMES = X_train.columns[:]


# dot_data = export_graphviz(Model1,
#                                out_file=None,
#                                feature_names=FEATURE_NAMES,
#                                filled = True)
# graph = Source(dot_data)

# # dot_data = io.StringIO()
# # export_graphviz(Model1, out_file=dot_data, rounded=True, filled=True)
# # filename = "tree.png"
# # pydotplus.graph_from_dot_data(dot_data.getvalue()).write_png(filename)
# # plt.figure(figsize=(300,100))
# # img = mpimg.imread(filename)
# # imgplot = plt.imshow(img)
# # plt.show()

# plt.figure(figsize=(120,50))
# plot_tree(Model1, feature_names=FEATURE_NAMES,fontsize=11, rounded=True)
# plt.show()

In [7]:
# Visualising the Random Forest Regression results 
  
# arange for creating a range of values 
# from min value of x to max  
# value of x with a difference of 0.01  
# between two consecutive values 
# X_grid = np.arange(X_test, max(x), 0.01)  
# X_ram
  
# reshape for reshaping the data into a len(X_grid)*1 array,  
# i.e. to make a column out of the X_grid value                   
# X_grid = X_grid.reshape((len(X_grid), 1)) 
  
# Scatter plot for original data 
# plt.scatter(x, y, color = 'blue')   
  
# plot predicted data 
# plt.plot(Y_pred_min_array,  
#          color = 'green')  
# plt.title('Random Forest Regression') 
# plt.xlabel('Position level') 
# plt.ylabel('Max_price') 
# plt.show()

## 7. Predicting the test set

In [10]:
Model_Final = RandomForestRegressor(n_estimators = 1500,criterion='mae',random_state = 1)
Model_Final.fit(X_train,Ya)
Y_pred_test = Model_Final.predict(X_test)

RESULTS = []
for i in range(len(data2)):
    temp_result = []
    temp_result.append(data2['id'][i])
    temp_result.append(Y_pred_test[i][0])
    temp_result.append(Y_pred_test[i][1])
    RESULTS.append(temp_result)

RESULTS = pd.DataFrame(RESULTS)
print(RESULTS)



         0            1            2
0    28807  1126.516693  1194.133517
1    22559   384.017723   401.133870
2    28647   791.800283   821.233980
3    22141   528.440970   546.200083
4    26116  2124.674897  2300.583543
5    27111  1412.425080  1463.226603
6    23420   620.004083   646.379980
7    21464   188.816333   195.450357
8    29405   735.152740   771.372193
9    27107   404.442920   413.626227
10   26141   961.245523   995.154737
11   25928  1654.770423  1747.151687
12   24845   714.803043   771.859653
13   28804   366.151860   378.754013
14   26772   606.658737   614.803347
15   27413   890.453257   923.237697
16    7261   879.023217   923.084873
17   31424   804.495977   825.724143
18    3940   426.477407   436.223060
19   16238   534.921883   552.548620
20   29407  1206.710200  1260.674793
21   24011   339.209607   358.428527
22    8542   399.156407   405.994427
23   14448   722.820683   738.584460
24   19611   243.508187   253.090063
25   30068  1196.247467  1256.934607
2

In [11]:
RESULTS.to_csv("/Users/Simon/Documents/GitHub/adana123/RESULTS.csv",header=False,index=False)